In [ ]:
import math
import numpy as np
import random
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
goal_state = 500
size_of_maze = 1000
episodes = 1000
discount = 0.99
alpha = 0.2
e_soft = 0.1

n = 8192
mean = 0
sigma = 1 / math.sqrt(n)

inputs = n
ouputs = 1

reward_bad = -1
reward_good = 0

In [ ]:
state_hrrs = np.random.normal(mean, sigma, size=[size_of_maze, n])

In [ ]:
def find_goal(state_hrr, start_location):
    state = start_location
    steps = 0
    while(state != goal_state):
        if(state == 0):
            decision = policy(nn.feedforward(state_hrrs[size_of_maze-1]), nn.feedforward(state_hrrs[state+1]), 0)
            if(decision == -1):
                state = size_of_maze-1
            else:
                state = state + decision
        elif(state == size_of_maze - 1):
            decision = policy(nn.feedforward(state_hrrs[state-1]), nn.feedforward(state_hrrs[0]), 0)
            if(decision == -1):
                state = state + decision
            else:
                state_prime = state
                state = 0
        else:
            decision = policy(nn.feedforward(state_hrrs[state-1]), nn.feedforward(state_hrrs[state+1]), 0)
            state = state + decision
        steps += 1
    return steps

In [ ]:
class NeuralNetwork:
    def __init__(self, inputs, outputs, discount, alpha):
        self.discount = discount
        self.alpha = alpha
        self.input = inputs
        self.output = outputs
        self.weights = np.random.randn(outputs, inputs)
        self.bias = 1
        
    def feedforward(self, X):
        self.output = np.dot(self.weights, X) + self.bias
        return self.output
    
    def backprop(self, state_prime_hrr, state_hrr, y):
        self.error = (y + self.discount * self.feedforward(state_hrr)) - self.feedforward(state_prime_hrr)
        self.weights = np.add(self.weights, (self.alpha * self.error * state_prime_hrr))

In [ ]:
def policy(a, b, rand_on):
    if((np.random.random() < e_soft) and (rand_on == 1)):
        return np.random.choice([-1, 1])
    if(a > b):
        return -1
    elif (b > a):
        return 1
    else:
        return np.random.choice([-1, 1])

In [ ]:
nn = NeuralNetwork(n, 1, discount, alpha)
state = random.randint(0, size_of_maze - 1)
state_prime = state

for x in range(episodes):
        while(state != goal_state):
            if(state == 0):
                decision = policy(nn.feedforward(state_hrrs[size_of_maze-1]), nn.feedforward(state_hrrs[state+1]), 1)
                if(decision == -1):
                    state_prime = state
                    state = size_of_maze-1
                else:
                    state_prime = state
                    state = state + decision
            elif(state == size_of_maze - 1):
                decision = policy(nn.feedforward(state_hrrs[state-1]), nn.feedforward(state_hrrs[0]), 1)
                if(decision == -1):
                    state_prime = state
                    state = state + decision
                else:
                    state_prime = state
                    state = 0
            else:
                decision = policy(nn.feedforward(state_hrrs[state-1]), nn.feedforward(state_hrrs[state+1]), 1)
                state_prime = state
                state = state + decision
            nn.backprop(state_hrrs[state_prime], state_hrrs[state], reward_bad)
        nn.backprop(state_hrrs[state_prime], state_hrrs[state], reward_good)
        state = random.randint(0, size_of_maze - 1)
        state_prime = state
        print("Episode", x+1, "done")
           

In [ ]:
value = np.zeros(size_of_maze)
for x in range(size_of_maze):
    value[x] = nn.feedforward(state_hrrs[x])
position = np.arange(size_of_maze)
plt.plot(position, value)

In [ ]:
find_goal(state_hrrs[400], 400)